In [1]:
import json

from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

import math

In [2]:
# Need to run this the first time using NLTK for stopwords & tokenize
import nltk
nltk.download('stopwords')
import nltk
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/soobing/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/soobing/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
stop_words = set(stopwords.words('english'))

In [4]:
with open("Data/all_cases_with_text.txt") as json_file:
            all_cases_info =  json.load(json_file)

In [5]:
len(all_cases_info)

200

In [6]:
#master_tf_dict will be a dict in the format {'case 1':[sorted lsit of top 500 words and counts], 'case2':list}
master_tf_dict={}

#raw_df_dict will be a dict in the format {'word1':'#ofdocs mentioned', 'word2':'#ofdocs mentioned'}
raw_df_dict = {}

for item in all_cases_info:
    item_text = item["case_text"]
    text_tokenized = word_tokenize(item_text)
    no_stop_text = [w.lower() for w in text_tokenized if not w in stop_words]
    
    #removing words that are shorter than 3 letters - this removes words like ',' '.' '(' etc
    filtered_text = [w for w in no_stop_text if len(w)>3]
      
    #create a list of just the unique words
    unique_words = set(filtered_text)
    
    #update the master_idf_dict. I.e., for each word contained in this court case, tick up the master idf dict by 1
    for uword in unique_words:
        try:
            curr_word_idf = raw_df_dict[uword] + 1
        except:
            curr_word_idf = 1
            
        raw_df_dict.update({uword:curr_word_idf})

    #create a dict in the format {'word':count, 'word2':count2}
    word_count_dict={}    
    for uword in unique_words:
        raw_count = filtered_text.count(uword)
        
        #only take words that have more than 1 mention
        if raw_count > 1:
            word_count_dict.update({uword:raw_count})
  
    master_tf_dict.update({item["case_code"]:word_count_dict})

In [7]:
#use raw_df_dict to calculate idf for each term
# idf = log [ (total # docs in corpus) / (total # docs that mention the word) ]
total_docs = len(master_tf_dict)
master_idf_dict={}

for key, value in raw_df_dict.items():
    docs_with_term = value
    idf = math.log(total_docs / docs_with_term)
    master_idf_dict.update({key:idf})

In [ ]:
#go through master_tf_dict, and search up idf for each of those terms, and then calculate tf*idf for each term.

master_tfidf_dict=[]
curr_case_num=0

for case, terms in master_tf_dict.items():
    curr_case_num+=1
    #we will put words and tfidf scores into case_tfidf_dict for each case, then update the master_tfidf_dict with that
    case_tfidf_dict={}
    
    for key, value in terms.items():
        word = key
        tf = value
        
        idf = master_idf_dict[word]
        print("word: "+word)
        print("tf: "+str(tf))
        print("idf: "+str(idf))
        
        tf_idf = tf * idf 
        print("tf_idf: "+str(tf_idf))
        
        #only taking items with tf_idf above 3 -> this eliminates a lot of entries.
        #previously the tf_idf dict was over 100000 lines
        if tf_idf > 3:
            case_tfidf_dict.update({word:tf_idf})
    
    #at this point in the code we have case_tfidf_dict which shows the tfidf of every word in the case we are iterating
    #through. 
    #create sorted dict so we can take the top 5 terms
    sorted_dict = sorted(case_tfidf_dict.items(), key=lambda case_tfidf_dict:case_tfidf_dict[1], reverse=True)
    
    bb = sorted_dict[:5]
    aa = []
    
    for a in bb:
        aa.append(a[0])
    
    #loop thru all_cases_info to find entry so we can take case name etc
    for item in all_cases_info:
        if item["case_code"] == case:
            curr_case = item
    
    master_tfidf_dict.append({
        "case_code":case,
        "case_name":curr_case["case_name"],
        "criminal_or_tax":curr_case["crim_or_tax"],
        "top_tfidf":aa,
        "url":curr_case["url"]})

word: legislative
tf: 10
idf: 0.9545119446943529
tf_idf: 9.545119446943529
word: march
tf: 4
idf: 1.0216512475319812
tf_idf: 4.086604990127925
word: instance
tf: 7
idf: 1.155182640156504
tf_idf: 8.086278481095528
word: s.a.c.
tf: 2
idf: 4.605170185988092
tf_idf: 9.210340371976184
word: eligible
tf: 2
idf: 2.0024805005437076
tf_idf: 4.004961001087415
word: though
tf: 9
idf: 0.7765287894989963
tf_idf: 6.988759105490967
word: test
tf: 2
idf: 0.5621189181535412
tf_idf: 1.1242378363070824
word: what
tf: 6
idf: 0.8324092478934529
tf_idf: 4.994455487360717
word: involve
tf: 3
idf: 1.3093333199837622
tf_idf: 3.9279999599512867
word: reading
tf: 13
idf: 1.07880966137193
tf_idf: 14.02452559783509
word: rosaire
tf: 3
idf: 5.298317366548036
tf_idf: 15.894952099644108
word: suggest
tf: 3
idf: 0.8209805520698303
tf_idf: 2.462941656209491
word: consistently
tf: 6
idf: 1.5141277326297755
tf_idf: 9.084766395778653
word: consequences
tf: 7
idf: 1.0216512475319812
tf_idf: 7.151558732723869
word: crim
tf:

idf: 2.120263536200091
tf_idf: 339.2421657920146
word: accordingly
tf: 7
idf: 0.21072103131565253
tf_idf: 1.4750472192095678
word: 2008
tf: 16
idf: 1.0498221244986776
tf_idf: 16.797153991978842
word: leads
tf: 3
idf: 1.40649706843741
tf_idf: 4.21949120531223
word: type
tf: 6
idf: 0.8556661100577203
tf_idf: 5.133996660346321
word: feature
tf: 2
idf: 2.407945608651872
tf_idf: 4.815891217303744
word: factors
tf: 8
idf: 0.8794767587514388
tf_idf: 7.035814070011511
word: exists
tf: 4
idf: 1.3093333199837622
tf_idf: 5.237333279935049
word: jurisprudence
tf: 5
idf: 1.035637489506721
tf_idf: 5.178187447533605
word: furthermore
tf: 4
idf: 1.0498221244986776
tf_idf: 4.1992884979947105
word: conferring
tf: 2
idf: 2.659260036932778
tf_idf: 5.318520073865556
word: sentenced
tf: 31
idf: 2.0794415416798357
tf_idf: 64.4626877920749
word: nonetheless
tf: 6
idf: 0.8915981192837835
tf_idf: 5.349588715702701
word: evasive
tf: 4
idf: 3.6888794541139363
tf_idf: 14.755517816455745
word: either
tf: 8
idf: 0.4

tf: 5
idf: 0.6348782724359695
tf_idf: 3.1743913621798474
word: begin
tf: 2
idf: 1.491654876777717
tf_idf: 2.983309753555434
word: also
tf: 24
idf: 0.23572233352106994
tf_idf: 5.657336004505678
word: confuse
tf: 2
idf: 2.8134107167600364
tf_idf: 5.626821433520073
word: alive
tf: 2
idf: 2.407945608651872
tf_idf: 4.815891217303744
word: respondent
tf: 30
idf: 0.03045920748470844
tf_idf: 0.9137762245412532
word: inspired
tf: 4
idf: 4.605170185988092
tf_idf: 18.420680743952367
word: maximum
tf: 5
idf: 1.9310215365615626
tf_idf: 9.655107682807813
word: laws
tf: 23
idf: 1.6873994539038122
tf_idf: 38.81018743978768
word: crime
tf: 9
idf: 1.40649706843741
tf_idf: 12.65847361593669
word: reiterate
tf: 3
idf: 2.5902671654458267
tf_idf: 7.77080149633748
word: granted
tf: 4
idf: 1.035637489506721
tf_idf: 4.142549958026884
word: adopted
tf: 12
idf: 0.8096809968158969
tf_idf: 9.716171961790764
word: actions
tf: 2
idf: 1.2552660987134865
tf_idf: 2.510532197426973
word: 1123
tf: 2
idf: 4.19970507787992

word: search
tf: 3
idf: 1.4271163556401458
tf_idf: 4.281349066920438
word: parliament
tf: 3
idf: 0.6831968497067772
tf_idf: 2.0495905491203317
word: clients
tf: 22
idf: 2.995732273553991
tf_idf: 65.9061100181878
word: establish
tf: 2
idf: 0.7031975164134469
tf_idf: 1.4063950328268937
word: persons
tf: 4
idf: 0.7444404749474958
tf_idf: 2.977761899789983
word: time
tf: 4
idf: 0.22941316432780512
tf_idf: 0.9176526573112205
word: seeking
tf: 3
idf: 1.07880966137193
tf_idf: 3.23642898411579
word: considers
tf: 2
idf: 1.40649706843741
tf_idf: 2.81299413687482
word: court
tf: 63
idf: 0.0
tf_idf: 0.0
word: prosecution
tf: 3
idf: 1.1874435023747254
tf_idf: 3.562330507124176
word: diminish
tf: 2
idf: 2.900422093749666
tf_idf: 5.800844187499332
word: situations
tf: 3
idf: 1.1239300966523995
tf_idf: 3.3717902899571985
word: based
tf: 2
idf: 0.33547273628812935
tf_idf: 0.6709454725762587
word: henry
tf: 2
idf: 2.659260036932778
tf_idf: 5.318520073865556
word: objective
tf: 4
idf: 1.0078579253996456

tf: 35
idf: 0.9545119446943529
tf_idf: 33.40791806430235
word: defect
tf: 4
idf: 3.506557897319982
tf_idf: 14.026231589279927
word: characterizes
tf: 2
idf: 3.6888794541139363
tf_idf: 7.3777589082278725
word: context
tf: 13
idf: 0.3930425881096072
tf_idf: 5.109553645424893
word: agreed
tf: 2
idf: 0.6348782724359695
tf_idf: 1.269756544871939
word: authorities
tf: 5
idf: 1.1874435023747254
tf_idf: 5.937217511873627
word: referred
tf: 3
idf: 0.28768207245178085
tf_idf: 0.8630462173553426
word: supreme
tf: 3
idf: 0.21072103131565253
tf_idf: 0.6321630939469576
word: societies
tf: 6
idf: 3.506557897319982
tf_idf: 21.03934738391989
word: third
tf: 7
idf: 0.6254885320861305
tf_idf: 4.378419724602914
word: indeed
tf: 2
idf: 0.6069694843188931
tf_idf: 1.2139389686377862
word: currently
tf: 2
idf: 2.407945608651872
tf_idf: 4.815891217303744
word: evidence
tf: 6
idf: 0.23572233352106994
tf_idf: 1.4143340011264196
word: appeal
tf: 26
idf: 0.0
tf_idf: 0.0
word: minister
tf: 26
idf: 0.606969484318893

...

idf: 4.605170185988092
tf_idf: 488.1480397147377
word: textual
tf: 7
idf: 2.5902671654458267
tf_idf: 18.131870158120787
word: reducing
tf: 6
idf: 2.2537949288246137
tf_idf: 13.522769572947682
word: involving
tf: 3
idf: 1.0498221244986776
tf_idf: 3.1494663734960326
word: arises
tf: 2
idf: 1.171182981502945
tf_idf: 2.34236596300589
word: includes
tf: 4
idf: 0.9416085398584451
tf_idf: 3.7664341594337802
word: report
tf: 3
idf: 1.07880966137193
tf_idf: 3.23642898411579
word: supp
tf: 6
idf: 1.3862943611198906
tf_idf: 8.317766166719343
word: individual
tf: 27
idf: 0.7874578600311867
tf_idf: 21.26136222084204
word: whole
tf: 9
idf: 0.7133498878774648
tf_idf: 6.420148990897183
word: object
tf: 25
idf: 1.0498221244986776
tf_idf: 26.24555311246694
word: recharacterized
tf: 2
idf: 3.6888794541139363
tf_idf: 7.3777589082278725
word: find
tf: 3
idf: 0.5361434317502806
tf_idf: 1.6084302952508418
word: automatically
tf: 5
idf: 1.8971199848858813
tf_idf: 9.485599924429406
word: frustrate
tf: 4
idf: 2

idf: 0.653926467406664
tf_idf: 2.615705869626656
word: finds
tf: 3
idf: 1.5606477482646683
tf_idf: 4.681943244794005
word: seriously
tf: 3
idf: 1.8643301620628903
tf_idf: 5.592990486188671
word: further
tf: 6
idf: 1.07880966137193
tf_idf: 6.47285796823158
word: nothing
tf: 4
idf: 0.5709295478356962
tf_idf: 2.2837181913427846
word: suspect
tf: 3
idf: 2.2537949288246137
tf_idf: 6.761384786473841
word: practicality
tf: 2
idf: 4.605170185988092
tf_idf: 9.210340371976184
word: ecstasy
tf: 5
idf: 3.912023005428146
tf_idf: 19.56011502714073
word: remained
tf: 2
idf: 1.3862943611198906
tf_idf: 2.772588722239781
word: operating
tf: 5
idf: 2.0402208285265546
tf_idf: 10.201104142632772
word: threshold
tf: 4
idf: 1.3470736479666094
tf_idf: 5.388294591866438
word: hodgson
tf: 5
idf: 3.912023005428146
tf_idf: 19.56011502714073
word: seize
tf: 28
idf: 2.3538783873815965
tf_idf: 65.90859484668471
word: options
tf: 5
idf: 2.207274913189721
tf_idf: 11.036374565948606
word: dereliction
tf: 3
idf: 5.29831

idf: 1.8643301620628903
tf_idf: 14.914641296503122
word: agree
tf: 8
idf: 0.30788477976930034
tf_idf: 2.4630782381544027
word: impossibility
tf: 3
idf: 3.6888794541139363
tf_idf: 11.06663836234181
word: entering
tf: 5
idf: 1.7719568419318754
tf_idf: 8.859784209659377
word: searching
tf: 2
idf: 3.2188758248682006
tf_idf: 6.437751649736401
word: easily
tf: 3
idf: 1.6873994539038122
tf_idf: 5.062198361711436
word: prevented
tf: 3
idf: 1.8971199848858813
tf_idf: 5.691359954657644
word: bulletproof
tf: 5
idf: 5.298317366548036
tf_idf: 26.491586832740182
word: difficilement
tf: 5
idf: 5.298317366548036
tf_idf: 26.491586832740182
word: account
tf: 4
idf: 0.44628710262841953
tf_idf: 1.7851484105136781
word: statute
tf: 2
idf: 0.843970070294529
tf_idf: 1.687940140589058
word: loss
tf: 4
idf: 1.155182640156504
tf_idf: 4.620730560626016
word: repute
tf: 2
idf: 3.506557897319982
tf_idf: 7.013115794639964
word: dismissed
tf: 2
idf: 0.24846135929849972
tf_idf: 0.49692271859699944
word: precluded
tf:

tf_idf: 3.4295968561838532
word: supp
tf: 5
idf: 1.3862943611198906
tf_idf: 6.931471805599453
word: individual
tf: 2
idf: 0.7874578600311867
tf_idf: 1.5749157200623733
word: 231.2
tf: 50
idf: 3.6888794541139363
tf_idf: 184.4439727056968
word: operates
tf: 2
idf: 1.7147984280919266
tf_idf: 3.4295968561838532
word: appendix
tf: 2
idf: 2.162823150618887
tf_idf: 4.325646301237774
word: relate
tf: 4
idf: 1.3470736479666094
tf_idf: 5.388294591866438
word: ottawa
tf: 2
idf: 0.7444404749474958
tf_idf: 1.4888809498949915
word: practice
tf: 3
idf: 1.0642108619507773
tf_idf: 3.1926325858523317
word: find
tf: 2
idf: 0.5361434317502806
tf_idf: 1.0722868635005611
word: justice
tf: 6
idf: 0.4155154439616658
tf_idf: 2.493092663769995
word: given
tf: 8
idf: 0.29437106060257756
tf_idf: 2.3549684848206205
word: expectation
tf: 4
idf: 1.8325814637483102
tf_idf: 7.330325854993241
word: demand
tf: 3
idf: 1.8018098050815565
tf_idf: 5.40542941524467
word: amount
tf: 7
idf: 0.5447271754416719
tf_idf: 3.8130902

tf: 36
idf: 0.7339691750802005
tf_idf: 26.422890302887218
word: namely
tf: 3
idf: 0.8209805520698303
tf_idf: 2.462941656209491
word: address
tf: 3
idf: 0.8209805520698303
tf_idf: 2.462941656209491
word: said
tf: 4
idf: 0.43078291609245434
tf_idf: 1.7231316643698174
word: reasonable
tf: 7
idf: 0.47803580094299974
tf_idf: 3.3462506066009983
word: sufficiently
tf: 2
idf: 1.1874435023747254
tf_idf: 2.374887004749451
word: appear
tf: 3
idf: 0.9038682118755978
tf_idf: 2.7116046356267933
word: non-medically
tf: 3
idf: 5.298317366548036
tf_idf: 15.894952099644108
word: conventional
tf: 5
idf: 3.2188758248682006
tf_idf: 16.094379124341003
word: chan
tf: 2
idf: 2.995732273553991
tf_idf: 5.991464547107982
word: well
tf: 3
idf: 0.4155154439616658
tf_idf: 1.2465463318849974
word: appellant
tf: 43
idf: 0.0460439385014068
tf_idf: 1.9798893555604924
word: with
tf: 2
idf: 0.5191938734365074
tf_idf: 1.0383877468730147
word: example
tf: 11
idf: 0.48613301117561913
tf_idf: 5.347463122931811
word: failed
t

In [14]:
master_tfidf_dict

[{'case_code': 'criminal2019SCC47',
  'case_name': 'R. v. Poulin',
  'criminal_or_tax': 'criminal',
  'top_tfidf': ['p, u, n, i, s',
   'p, o, u, l, i',
   's, e, n, t, e',
   'o, f, f, e, n',
   'g, l, o, b, a'],
  'url': 'https://scc-csc.lexum.com/scc-csc/scc-csc/en/item/17964/index.do'},
 {'case_code': 'tax[2016] 1 SCR 336',
  'case_name': 'Canada (Attorney General) v. Chambre des notaires du Québec',
  'criminal_or_tax': 'tax',
  'top_tfidf': ['s, e, c, r, e',
   'p, r, o, f, e',
   'n, o, t, a, r',
   'n, o, t, a, r',
   'c, l, i, e, n'],
  'url': 'https://scc-csc.lexum.com/scc-csc/scc-csc/en/item/15989/index.do'}]

In [ ]:
master_tfidf_dict[0]['top_tfidf'][0][0]

In [ ]:
##printing just the tfidf scores to text
with open("Data/data.text", 'w') as outfile:
    json.dump(master_tfidf_dict, outfile)

In [ ]:
bb = sorted_dict[:5]
bb

In [ ]:
keywords = f'{bb[0][0]}, {bb[1][0]}, {bb[2][0]}, {bb[3][0]}, {bb[4][0]}'
keywords

In [ ]:
bb = sorted_dict[:5]
aa = []
for a in bb:
    aa.append(a[0])
aa

In [ ]:
cc = []
for a in aa:
    to_string = f'{aa[0]}, {aa[1]}, {aa[2]}, {aa[3]}, {aa[4]}'
    cc.append(to_string)
cc

In [ ]:
##adding the tfidf scores to the all_cases_with_text.txt file, and creating "all_cases_text_and_tfidf.txt"
with open("Data/all_cases_with_text.txt") as json_file:
    all_cases_with_text =  json.load(json_file)